# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import json
from PIL import Image

import torch

from torchvision import transforms, datasets, models
from torch import nn, optim
from network import Network
import torch.nn.functional as F

/home/jan/.local/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
load_model_from_checkpoint = False
run_trainning = True
train_executed = False

checkpoint_path = './checkpoints/densenet201_dropout0_5.pth'

device ='cpu'
if torch.cuda.is_available:
    device = 'cuda'

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [3]:
data_dir = './flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [4]:
# TODO: Define your transforms for the training and validation sets
means = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {'training' : transforms.Compose([transforms.Resize(256),
                                       transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(means, std)]),
                  'testing' : transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(means, std)]),

                  'validation' : transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(means, std)])                                     
                  }

# DONE: Load the datasets with ImageFolder
image_datasets = {'training' : datasets.ImageFolder(train_dir,transform=data_transforms['training']),
                  'validation' : datasets.ImageFolder(valid_dir,transform=data_transforms['validation']),
                  'testing' : datasets.ImageFolder(test_dir,transform=data_transforms['testing'])
                 }

# DONE: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {'training' : torch.utils.data.DataLoader(image_datasets['training'],batch_size= 128, shuffle= True),
               'validation' : torch.utils.data.DataLoader(image_datasets['validation'],batch_size= 32, shuffle= True),
               'testing' : torch.utils.data.DataLoader(image_datasets['testing'],batch_size= 32, shuffle= True)
              }

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [5]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [6]:
class_to_index = image_datasets['training'].class_to_idx
class_from_index = dict([val,key] for key,val in class_to_index.items())

print(class_from_index)

{0: '1', 1: '10', 2: '100', 3: '101', 4: '102', 5: '11', 6: '12', 7: '13', 8: '14', 9: '15', 10: '16', 11: '17', 12: '18', 13: '19', 14: '2', 15: '20', 16: '21', 17: '22', 18: '23', 19: '24', 20: '25', 21: '26', 22: '27', 23: '28', 24: '29', 25: '3', 26: '30', 27: '31', 28: '32', 29: '33', 30: '34', 31: '35', 32: '36', 33: '37', 34: '38', 35: '39', 36: '4', 37: '40', 38: '41', 39: '42', 40: '43', 41: '44', 42: '45', 43: '46', 44: '47', 45: '48', 46: '49', 47: '5', 48: '50', 49: '51', 50: '52', 51: '53', 52: '54', 53: '55', 54: '56', 55: '57', 56: '58', 57: '59', 58: '6', 59: '60', 60: '61', 61: '62', 62: '63', 63: '64', 64: '65', 65: '66', 66: '67', 67: '68', 68: '69', 69: '7', 70: '70', 71: '71', 72: '72', 73: '73', 74: '74', 75: '75', 76: '76', 77: '77', 78: '78', 79: '79', 80: '8', 81: '80', 82: '81', 83: '82', 84: '83', 85: '84', 86: '85', 87: '86', 88: '87', 89: '88', 90: '89', 91: '9', 92: '90', 93: '91', 94: '92', 95: '93', 96: '94', 97: '95', 98: '96', 99: '97', 100: '98', 101:

In [7]:
def get_class_name(class_from_index):
    return cat_to_name[class_from_index]

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [8]:
# TODO: Build and train your network
densenet = models.densenet201(pretrained=True)

/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/model

/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/model

/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
/home/jan/anaconda3/envs/PytorchDL/lib/python3.6/site-packages/torchvision/model

In [9]:
# Check the densenet layers
densenet

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [10]:
# Freeze layers
for param in densenet.parameters():
    param.requires_grad = False

In [11]:
# Input size of our network should match the output of the pre-trained network
classifier_input_size = densenet.classifier.in_features
print(classifier_input_size)

1920


In [12]:
#resnet = models.resnet50(pretrained=True)

In [13]:
#resnet.fc.in_features
#checkresnetlayers
#resnet

In [14]:
#def create_classifier():
#    classifier = nn.Sequential(
#        nn.Linear(classifier_input_size,hidden_units),
#        nn.Dropout(p=dropout_p),    
#        nn.ReLU(),        
#        nn.Linear(hidden_units,102),
#        nn.ReLU(), 
#        nn.Dropout(p=dropout_p),
#        nn.LogSoftmax(dim = 1))
#    return classifier

In [15]:
input_size = 1920
hidden_sizes = [512]
output_size = 102
dropout = 0.5
learning_rate = 0.001

classifier = Network(input_size,hidden_sizes,output_size,dropout)
densenet.classifier = classifier
densenet.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [16]:
def test_accuracy(model,dataloader,criterion):
    current_loss = 0
    accuracy = 0
    model.eval()
    
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)       
        
        output = model.forward(images)
        current_loss += criterion(output,labels).item()
        
        ps = torch.exp(output)
                   
        # Each probability consist in an array of probabilities, the predicted class is the
        # highest probability of each one, so we need to get the max of the probabilitie for each 
        # prediction in order to find the predicted_class         
        
        predicted_classes = ps.max(dim=1)[1]
        
        # To find the correct classified items we compare labels with the predicted_classes
        # and we will get back an array of booleans with 1 if the prediction was correct or
        # 0 otherwise
                
        correct_predictions = (labels.data == predicted_classes)

        accuracy += correct_predictions.type(torch.FloatTensor).mean()
        
    return current_loss, accuracy

In [17]:
criterion = nn.NLLLoss()
adam_optimizer = optim.Adam(densenet.classifier.parameters(), lr = learning_rate)

In [18]:
def train_network(model):
# Train Network
    epochs = 10
    print_every = 20
    step = 0
    running_loss = 0
    validation_data = dataloaders['validation']
    len_val = len(validation_data)

    model = model.to(device)    

    
    for epoch in range(epochs):
        running_loss = 0    
        model.train()

        for inputs, y in dataloaders['training']:
            step += 1
            adam_optimizer.zero_grad()
            inputs, y = inputs.to(device), y.to(device)

            y_hat = model.forward(inputs)

            loss = criterion(y_hat,y)
            loss.backward()

            adam_optimizer.step()

            running_loss += loss.item()

            if step %  print_every == 0:
                model.eval()

                with torch.no_grad():
                    test_loss, accuracy = test_accuracy(model,validation_data,criterion)

                print("Epoch: {}/{}... ".format(epoch+1, epochs),
                      "Loss: {:.3f}".format(running_loss/print_every),
                      "Test Loss: {:.3f}".format(test_loss/len_val),
                      "Test Accuracy: {:.3f}".format(accuracy/len_val))

                running_loss = 0
                model.train()

    train_executed = True

## Testing Network

In [19]:
# TODO: Do validation on the test set
model_validated = False
def validate_trained_model(dataloader):
    _, accuracy = test_accuracy(densenet,dataloader,criterion)
    print('Accuracy in the validation set is: {} %'.format(round(float(accuracy/len(dataloader)),2)*100))
    #print('Accuracy in the validation set is: {} '.format(accuracy/len(dataloaders['validation'])))
    
# DONE: Do validation on the test set
if train_executed:    
    validate_trained_model(dataloaders['testing'])
    model_validated =True
else:
    print('Model was not trained yet, check the model accuracy below after loading the checkpoint')

Model was not trained yet, check the model accuracy below after loading the checkpoint


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [20]:
# TODO: Save the checkpoint 
def save_checkpoint():
    checkpoint = {'class_from_index' : class_from_index,
                  'class_to_index' : class_to_index,
                  'hidden_units' : hidden_units,
                  'learning_rate' : learning_rate,
                  'batch_size' : batch_size,
                  'device' : device,
                  'state_dict' : densenet.state_dict()}
    torch.save(checkpoint, checkpoint_path)
    print('Checkpoint saved')

In [21]:
if run_trainning:
    train_network(densenet)
    save_checkpoint()

Epoch: 1/10...  Loss: 4.370 Test Loss: 3.890 Test Accuracy: 0.188
Epoch: 1/10...  Loss: 3.641 Test Loss: 3.031 Test Accuracy: 0.380
Epoch: 2/10...  Loss: 1.068 Test Loss: 2.253 Test Accuracy: 0.520
Epoch: 2/10...  Loss: 2.345 Test Loss: 1.657 Test Accuracy: 0.713
Epoch: 2/10...  Loss: 1.964 Test Loss: 1.318 Test Accuracy: 0.767
Epoch: 3/10...  Loss: 1.340 Test Loss: 1.068 Test Accuracy: 0.808
Epoch: 3/10...  Loss: 1.441 Test Loss: 0.882 Test Accuracy: 0.827
Epoch: 4/10...  Loss: 0.254 Test Loss: 0.818 Test Accuracy: 0.849
Epoch: 4/10...  Loss: 1.275 Test Loss: 0.720 Test Accuracy: 0.860
Epoch: 4/10...  Loss: 1.123 Test Loss: 0.631 Test Accuracy: 0.884
Epoch: 5/10...  Loss: 0.636 Test Loss: 0.580 Test Accuracy: 0.881
Epoch: 5/10...  Loss: 0.978 Test Loss: 0.515 Test Accuracy: 0.904
Epoch: 5/10...  Loss: 0.992 Test Loss: 0.506 Test Accuracy: 0.891
Epoch: 6/10...  Loss: 0.952 Test Loss: 0.485 Test Accuracy: 0.901
Epoch: 6/10...  Loss: 0.911 Test Loss: 0.432 Test Accuracy: 0.900
Epoch: 7/1

NameError: name 'hidden_units' is not defined

In [ ]:
# DONE: Save the checkpoint 
if train_executed:    
    save_checkpoint()

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# DONE: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(path):
    # Load checkpoint
    checkpoint = torch.load(path)        
    
    hidden_units = checkpoint['hidden_units']
    learning_rate = checkpoint['learning_rate']
    batch_size = checkpoint['batch_size']
    class_to_index = checkpoint['class_to_index']
    
    # Rebuild Model
    classifier = create_classifier()
    densenet = models.densenet161(pretrained=True)
    densenet.classifier = classifier
    densenet.to(checkpoint['device'])
    
    densenet.load_state_dict(checkpoint['state_dict'])
    class_from_index = checkpoint['class_from_index']

In [ ]:
if load_model_from_checkpoint:
    load_checkpoint(checkpoint_path)
    print('Checkpoint loaded!')
else:
    print('Starting from scratch')

In [ ]:
if not model_validated:
    validate_trained_model(dataloaders['testing'])
    model_validated = True
else:
    print('Model was already validated')

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
# DONE: Process a PIL image for use in a PyTorch model
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    img = Image.open(image)
    
    transformations = transforms.Compose([transforms.Resize(256),
                        transforms.CenterCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize(means, std)])
    img = transformations(img)
    
    np_image = np.array(img) 
    
    
    return np_image

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
test_paths = [
    'flowers/valid/1/image_06739.jpg',
    'flowers/valid/1/image_06749.jpg',
    'flowers/valid/43/image_02336.jpg',
    'flowers/valid/10/image_07094.jpg',
'flowers/valid/10/image_07101.jpg',
'flowers/valid/10/image_07102.jpg',
'flowers/valid/10/image_07107.jpg',

'flowers/valid/100/image_07895.jpg',
'flowers/valid/100/image_07904.jpg',
'flowers/valid/100/image_07905.jpg',
'flowers/valid/100/image_07917.jpg',
'flowers/valid/100/image_07929.jpg',

'flowers/valid/101/image_07951.jpg',
'flowers/valid/101/image_07962.jpg',
'flowers/valid/101/image_07963.jpg',
'flowers/valid/101/image_07970.jpg',
'flowers/valid/101/image_07985.jpg']

imshow(process_image(test_paths[1]))

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
# DONE: Implement the code to predict the class from an image file
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    image = process_image(image_path)
    image = torch.from_numpy(image)
    image = image.to(device)
    
    model.eval()
    
    image.unsqueeze_(0)
    
    output = model.forward(image)
    
    ps = torch.exp(output)    
        
    probs, classes = ps.topk(topk)        
    probs, classes = probs.cpu().detach().numpy(),classes.cpu().detach().numpy()
    
    return probs , classes   

In [ ]:
print(class_to_index['1'])
print(class_to_index['43'])

In [ ]:
# DONE: Checks

ps, classes = predict(paths_1[0],densenet)

#classes = classes.cpu().numpy()
correct_precition = 0 in classes

print('Prediction is correct: {} with classes {}'.format(correct_precition,classes))

ps, classes = predict(paths_1[1],densenet)
#classes = classes.cpu().numpy()
correct_precition = 0 in classes

print('Prediction is correct: {} with classes {}'.format(correct_precition,classes))

ps, classes = predict(paths_43[0],densenet)
#classes = classes.cpu().numpy()
correct_precition = 40 in classes

print('Prediction is correct: {} with classes {}'.format(correct_precition,classes))

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
def display_prediction_probs(path, model):
    ps, classes = predict(path, model)
    
    # Create labels with category names for the topK predictions    
    categories = [cat_to_name[class_from_index[idx]] for idx in classes[0]] # [cat_to_name[idx] for idx in classes]       
            
    # Get true label
    true_idx = path.split('/')[2]
    true_label = cat_to_name[true_idx]
    
    # Show image with correct label
    fig, ax = plt.subplots(2,1)    
    
    imshow(process_image(path),title=true_label, ax = ax[0])    
        
    ax[1] = sb.barplot(x= ps[0], y= categories, color= sb.color_palette()[0])    
    #ax.plot()
    
    # Plot predictions with labels
    
    plt.show() 

In [ ]:
def display_prediction(path, model):
    # Get true label
    true_idx = path.split('/')[2]
    true_label = cat_to_name[true_idx]
    
    imshow(process_image(path),title=true_label)
    display_prediction_probs(path, model)

In [ ]:
for path in test_paths:
    display_prediction_probs(path,densenet)

In [ ]:
display_prediction_probs(paths_1[1],densenet)

In [ ]:
display_prediction_probs(paths_43[0],densenet)

In [ ]:
idxs = np.array([50,  0,  6, 45, 90])

#print(type(a))
labels = {50:'fifty', 0:'zero', 6:'six' , 45:'forty five',90:'ninety'}
print([cat_to_name[class_from_index[idx]] for idx in idxs ])


In [ ]:
type(classes)

In [ ]:
categories = [cat_to_name[class_from_index[idx]] for idx in classes[0]] # [cat_to_name[idx] for idx in classes]

In [ ]:
categories

In [ ]:
probs =  [ 0.3366361,   0.24463563,  0.10304732,  0.08841886 , 0.0409045 ]
cats = ['a','b','c','d','e']
sb.barplot(x= probs, y= cats,color= sb.color_palette()[0])